In [1]:
import cv2
import numpy as np
import os
import yaml
from yaml.loader import SafeLoader

In [2]:
# load yaml
with open('data.yaml', mode='r') as f:
    data_yaml = yaml.load(f, Loader=SafeLoader)

labels = data_yaml['names']
labels

['train']

In [3]:
  #load YOLO Model

yolo = cv2.dnn.readNetFromONNX('Model/weights/best.onnx')
yolo.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
yolo.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

In [4]:
# load image
img = cv2.imread('02.jpg')
image = img.copy()

row, col, d = image.shape

#create blank square black matrix
max_rc = max(row, col) 
input_image = np.zeros((max_rc, max_rc, 3), dtype=np.uint8) 
input_image[0:row, 0:col] = image #overlay image

# get YOLO predictions

# returns a 4-dimensional NumPy array ("blob") 
# represents the preprocessed input image. 
# suitable for passing to a deep neural network model for inference.
INPUT_WH = 640
blob = cv2.dnn.blobFromImage(input_image, 1/255, (INPUT_WH, INPUT_WH), swapRB=True, crop=False)
yolo.setInput(blob)
preds = yolo.forward() # get predictions



In [5]:
print(preds.shape)

(1, 25200, 6)


In [6]:
# remove duplicates of 25200

# 1. filter detections based on confidence(0.4) and probablity score(0.25)
detections = preds[0]
boxes = []
confidences = []
classes = []

image_w, image_h = input_image.shape[:2]

# since square, both r same
x_factor = image_w / INPUT_WH
y_factor = image_h / INPUT_WH

# column names: center_x, center_y, w, h, confidence, prob-scores of classes
for i in range(len(detections)):
    row = detections[i]
    confidence = row[4] #4th columnn of ith row
    
    if confidence > 0.1:
        class_score = row[5:].max() # maximum probability of class detected
        class_id = row[5:].argmax() # index at which max prob occurs

        if class_score > 0.25:
            cx, cy, w, h = row[0:4]

            #construct bound box from these values
            left = int((cx - 0.5*w) * x_factor)
            top  = int((cy - 0.5*h) * y_factor)
            width = int(w * x_factor)
            height = int(h * y_factor)
            box = np.array([left, top, width, height])

            confidences.append(confidence)
            boxes.append(box)
            classes.append(class_id)

#clean
boxes_np = np.array(boxes).tolist()
confidences_np = np.array(confidences).tolist()

# Non Maximum Suppression
index = cv2.dnn.NMSBoxes(boxes_np, confidences_np, 0.1, 0.3)


In [7]:
index

array([5])

In [8]:
# extracy bounding box info

for ind in index:
    x, y, w, h = boxes_np[ind]
    bb_conf = int(confidences_np[ind]*100)
    class_id = classes[ind]
    class_name = labels[class_id]

    text = f'{class_name}: {bb_conf}%'
    
    cv2.rectangle(image, (x,y), (x+w, y+h), (0, 0, 255), 2)
    cv2.putText(image, text, (x, y-10), cv2.FONT_HERSHEY_PLAIN, 0.7, (0, 0, 255), 1)
    

In [9]:
#display

cv2.imshow('image', image)
cv2.waitKey(0)
cv2.destroyAllWindows()